In [1]:
import os
import sys
import json
import time
import hashlib
import pickle
import warnings
from datetime import datetime, timedelta
from typing import List, Dict, Tuple, Optional, Any
from dataclasses import dataclass, asdict
from concurrent.futures import ThreadPoolExecutor, as_completed
import re

import pandas as pd
import numpy as np
from scipy import stats
from sklearn.covariance import LedoitWolf
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import cvxpy as cp
import yfinance as yf
from tqdm import tqdm

# OpenRouter (using OpenAI SDK with custom endpoint)
import openai

# Interactive widgets
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, Layout, VBox, HBox, Tab, Output
from IPython.display import display, clear_output, HTML, Markdown

# Suppress warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Packages installed and imported successfully!")


class ConfigManager:
    """Manage all configurations for the LLM Council Portfolio Optimizer"""

    DEFAULT_CONFIG = {
        "project": {
            "name": "LLM Council Portfolio Optimizer",
            "start_date": "2020-01-01",
            "end_date": "2024-01-31",
            "in_sample_end": "2022-12-31",
            "universe_sizes": [5, 10, 15, 20],
            "risk_free_rate": 0.02,
            "benchmark": "SPY"
        },
        "llm": {
            "provider": "openrouter",
            "models": [
                # FREE Models (no cost)
                {"name": "Llama 3.2 3B", "id": "meta-llama/llama-3.2-3b-instruct:free", "enabled": True, "weight": 1.0, "cost": "free"},
                {"name": "Mistral 7B", "id": "mistralai/mistral-7b-instruct:free", "enabled": True, "weight": 1.0, "cost": "free"},
                {"name": "Gemma 7B", "id": "google/gemma-7b-it:free", "enabled": True, "weight": 1.0, "cost": "free"},
                {"name": "Qwen 2.5 7B", "id": "qwen/qwen-2.5-7b-instruct:free", "enabled": True, "weight": 1.0, "cost": "free"},
                {"name": "Phi-3 Mini", "id": "microsoft/phi-3-mini-128k-instruct:free", "enabled": True, "weight": 1.0, "cost": "free"},
                {"name": "Nous Hermes", "id": "nousresearch/nous-hermes-2-mixtral-8x7b-dpo:free", "enabled": False, "weight": 1.0, "cost": "free"},

                # Low-cost Models (your $10 credit can handle these)
                {"name": "GPT-3.5 Turbo", "id": "openai/gpt-3.5-turbo", "enabled": False, "weight": 1.2, "cost": "low"},
                {"name": "Claude Haiku", "id": "anthropic/claude-3-haiku", "enabled": False, "weight": 1.2, "cost": "low"},
                {"name": "Llama 3 70B", "id": "meta-llama/llama-3-70b-instruct", "enabled": False, "weight": 1.3, "cost": "medium"},
                {"name": "Mixtral 8x7B", "id": "mistralai/mixtral-8x7b-instruct", "enabled": False, "weight": 1.3, "cost": "medium"}
            ],
            "iterations": 2,
            "temperature": 0.3,
            "max_tokens": 500,
            "cache_enabled": True
        },
        "optimization": {
            "strategies": [
                {"name": "Equal Weight", "enabled": True},
                {"name": "Mean-Variance", "enabled": True},
                {"name": "Max Sharpe", "enabled": True},
                {"name": "Min Variance", "enabled": True},
                {"name": "Risk Parity", "enabled": True},
                {"name": "LLM Weighted", "enabled": True}
            ],
            "transaction_cost": 0.001,
            "max_turnover": 0.20,
            "sector_limit": 0.40
        },
        "analysis": {
            "metrics": ["sharpe", "return", "volatility", "max_dd", "win_rate"],
            "out_of_sample_periods": ["2023-01-01", "2023-07-01", "2024-01-01"],
            "rebalance_frequency": "quarterly",
            "confidence_level": 0.95
        }
    }

    def __init__(self, config_file: str = None):
        self.config = self.DEFAULT_CONFIG.copy()
        if config_file and os.path.exists(config_file):
            self.load_config(config_file)

    def load_config(self, config_file: str):
        """Load configuration from file"""
        with open(config_file, 'r') as f:
            self.config.update(json.load(f))

    def save_config(self, config_file: str):
        """Save configuration to file"""
        with open(config_file, 'w') as f:
            json.dump(self.config, f, indent=2)

    def get_enabled_models() -> List[Dict]:
        """Get list of enabled LLM models"""
        return [model for model in config_manager.config["llm"]["models"] if model["enabled"]]

    def get_enabled_strategies(self) -> List[str]:
        """Get list of enabled optimization strategies"""
        return [strategy["name"] for strategy in self.config["optimization"]["strategies"] if strategy["enabled"]]

    def update_model_status(self, model_name: str, enabled: bool):
        """Update model enabled status"""
        for model in self.config["llm"]["models"]:
            if model["name"] == model_name:
                model["enabled"] = enabled
                break

    def update_strategy_status(self, strategy_name: str, enabled: bool):
        """Update strategy enabled status"""
        for strategy in self.config["optimization"]["strategies"]:
            if strategy["name"] == strategy_name:
                strategy["enabled"] = enabled
                break

config_manager = ConfigManager()


@dataclass
class LLMResponse:
    """Data class for LLM responses"""
    model_name: str
    model_id: str
    prompt: str
    response: str
    tokens_used: int
    timestamp: datetime
    latency: float
    cost: float = 0.0
    error: Optional[str] = None

    def to_dict(self):
        return asdict(self)


class LLMCouncilMember:
    """Individual LLM member in the council"""

    def __init__(self, name: str, model_id: str, weight: float = 1.0, cost_type: str = "free"):
        self.name = name
        self.model_id = model_id
        self.weight = weight
        self.cost_type = cost_type
        self.responses = []
        self.successful_queries = 0
        self.failed_queries = 0
        self.total_tokens = 0
        self.total_latency = 0.0
        self.total_cost = 0.0

        # Initialize OpenAI client for OpenRouter
        self.client = None

    def initialize_client(self, api_key: str):
        """Initialize the OpenRouter client"""
        try:
            self.client = openai.OpenAI(
                base_url="https://openrouter.ai/api/v1",
                api_key=api_key
            )
            return True
        except Exception as e:
            print(f"Failed to initialize client for {self.name}: {e}")
            return False

    def query(self, prompt: str, temperature: float = 0.3, max_tokens: int = 500) -> LLMResponse:
        """Query this specific LLM"""
        if not self.client:
            return LLMResponse(
                model_name=self.name,
                model_id=self.model_id,
                prompt=prompt,
                response="",
                tokens_used=0,
                timestamp=datetime.now(),
                latency=0,
                cost=0.0,
                error="Client not initialized"
            )

        start_time = time.time()

        try:
            response = self.client.chat.completions.create(
                model=self.model_id,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=max_tokens,
                temperature=temperature
            )

            latency = time.time() - start_time
            response_text = response.choices[0].message.content
            tokens_used = response.usage.total_tokens if response.usage else 0

            # Estimate cost (approximate)
            cost = self._estimate_cost(tokens_used)

            llm_response = LLMResponse(
                model_name=self.name,
                model_id=self.model_id,
                prompt=prompt,
                response=response_text,
                tokens_used=tokens_used,
                timestamp=datetime.now(),
                latency=latency,
                cost=cost
            )

            self.successful_queries += 1
            self.total_tokens += tokens_used
            self.total_latency += latency
            self.total_cost += cost
            self.responses.append(llm_response)

            return llm_response

        except Exception as e:
            latency = time.time() - start_time
            self.failed_queries += 1

            return LLMResponse(
                model_name=self.name,
                model_id=self.model_id,
                prompt=prompt,
                response="",
                tokens_used=0,
                timestamp=datetime.now(),
                latency=latency,
                cost=0.0,
                error=str(e)
            )

    def _estimate_cost(self, tokens: int) -> float:
        """Estimate cost based on model type"""
        if "free" in self.model_id:
            return 0.0
        elif "gpt-3.5" in self.model_id or "haiku" in self.model_id:
            return tokens * 0.000001  # Approx $0.001 per 1K tokens
        else:
            return tokens * 0.000002  # Approx $0.002 per 1K tokens

    def get_stats(self) -> Dict:
        """Get statistics for this council member"""
        total_queries = self.successful_queries + self.failed_queries
        return {
            "name": self.name,
            "model_id": self.model_id,
            "successful_queries": self.successful_queries,
            "failed_queries": self.failed_queries,
            "success_rate": self.successful_queries / total_queries if total_queries > 0 else 0,
            "total_tokens": self.total_tokens,
            "avg_latency": self.total_latency / max(1, self.successful_queries),
            "total_cost": self.total_cost,
            "weight": self.weight,
            "cost_type": self.cost_type
        }


class PromptEngineer:
    """Handle prompt creation and management"""

    # Default prompt templates
    DEFAULT_PROMPTS = {
        "stock_selection_simple": """Select {num_stocks} stocks from the S&P 500 that you believe will perform well in the next 12 months.\nConsider factors like financial health, growth potential, and market position.\nReturn ONLY a comma-separated list of ticker symbols.\nExample: AAPL, MSFT, GOOGL""",

        "stock_selection_detailed": """As a financial analyst, select {num_stocks} stocks from the S&P 500 for a long-term investment portfolio.\n\nCRITERIA:\n1. Strong Fundamentals (ROE > 15%, Debt/Equity < 1)\n2. Consistent Revenue Growth (> 5% YoY)\n3. Competitive Advantage (moat, brand, patents)\n4. Reasonable Valuation (P/E < sector average)\n5. Good Management (consistent dividend, buybacks)\n\nSector diversification is important. Avoid overly concentrated sectors.\n\nReturn ONLY ticker symbols in alphabetical order, comma-separated.""",

        "stock_selection_thematic": """Select {num_stocks} stocks aligned with these investment themes:\n- Artificial Intelligence & Cloud Computing\n- Renewable Energy & Sustainability\n- Healthcare Innovation\n- Financial Technology\n- E-commerce & Digital Transformation\n\nFocus on companies leading in these areas with strong growth potential.\nReturn ONLY ticker symbols, comma-separated.""",

        "weight_assignment_basic": """Given these stocks: {tickers}\nAssign portfolio weights (0-100%) that sum to 100%.\nConsider your conviction level for each stock.\n\nFormat each line as: TICKER: WEIGHT%\nExample:\nAAPL: 15%\nMSFT: 12%\nGOOGL: 10%""",

        "weight_assignment_advanced": """For these stocks: {tickers}\nAssign portfolio weights using these principles:\n\n1. Higher conviction = higher weight (max 30% per stock)\n2. Consider diversification across sectors\n3. Balance between growth and stability\n4. Account for correlation risk\n\nReturn weights in this format:\nTICKER1: WEIGHT1%\nTICKER2: WEIGHT2%"""
    }

    def __init__(self):
        self.custom_prompts = {}

    def create_stock_selection_prompt(self, num_stocks: int, prompt_type: str = "simple", custom_prompt: str = None) -> str:
        """Create stock selection prompt"""
        if custom_prompt:
            return custom_prompt.format(num_stocks=num_stocks)

        if prompt_type == "simple":
            return self.DEFAULT_PROMPTS["stock_selection_simple"].format(num_stocks=num_stocks)
        elif prompt_type == "detailed":
            return self.DEFAULT_PROMPTS["stock_selection_detailed"].format(num_stocks=num_stocks)
        elif prompt_type == "thematic":
            return self.DEFAULT_PROMPTS["stock_selection_thematic"].format(num_stocks=num_stocks)
        else:
            return self.DEFAULT_PROMPTS["stock_selection_simple"].format(num_stocks=num_stocks)

    def create_weight_assignment_prompt(self, tickers: List[str], prompt_type: str = "basic", custom_prompt: str = None) -> str:
        """Create weight assignment prompt"""
        ticker_str = ", ".join(tickers[:15])  # Limit for readability

        if custom_prompt:
            return custom_prompt.format(tickers=ticker_str)

        if prompt_type == "basic":
            return self.DEFAULT_PROMPTS["weight_assignment_basic"].format(tickers=ticker_str)
        elif prompt_type == "advanced":
            return self.DEFAULT_PROMPTS["weight_assignment_advanced"].format(tickers=ticker_str)
        else:
            return self.DEFAULT_PROMPTS["weight_assignment_basic"].format(tickers=ticker_str)

    def extract_tickers(self, text: str) -> List[str]:
        """Extract ticker symbols from text"""
        text = text.upper()

        # Common non-ticker words to exclude
        exclude_words = {
            'AND', 'THE', 'FOR', 'WITH', 'FROM', 'THAT', 'THIS', 'HAVE', 'WILL',
            'WHICH', 'THEIR', 'BEEN', 'STOCKS', 'TICKERS', 'INCLUDE', 'RECOMMEND',
            'PORTFOLIO', 'EXAMPLE', 'RETURN', 'ONLY', 'LIST', 'SELECT', 'CHOOSE'
        }

        # Remove explanation lines
        lines = text.split('\n')
        clean_lines = []

        for line in lines:
            line = line.strip()
            if not line:
                continue
            # Skip lines that look like explanations
            if any(phrase in line for phrase in [':', 'because', 'due to', 'since', 'as']):
                continue
            # Skip lines starting with numbers or bullets
            if re.match(r'^[\d•\-]', line):
                continue
            clean_lines.append(line)

        clean_text = ' '.join(clean_lines)

        # Extract tickers (1-5 uppercase letters)
        ticker_pattern = r'\b[A-Z]{1,5}\b'
        potential_tickers = re.findall(ticker_pattern, clean_text)

        # Filter valid tickers
        valid_tickers = []
        for ticker in potential_tickers:
            if (
                ticker not in exclude_words and
                len(ticker) >= 2 and
                ticker not in valid_tickers
            ):
                valid_tickers.append(ticker)

        return valid_tickers[:20]  # Limit to 20

    def extract_weights(self, text: str) -> Dict[str, float]:
        """Extract ticker-weight pairs from text"""
        weights = {}
        text = text.upper()

        # Multiple patterns to match different formats
        patterns = [
            r'([A-Z]{1,5})\s*[:=]\s*([\d.]+)\s*%?',  # AAPL: 15% or AAPL = 15
            r'([A-Z]{1,5})\s+([\d.]+)\s*%',  # AAPL 15%
            r'([A-Z]{1,5})\s*\(([\d.]+)%\)',  # AAPL (15%)
        ]

        for pattern in patterns:
            matches = re.findall(pattern, text)
            for ticker, weight in matches:
                try:
                    weight_float = float(weight)
                    if 0 < weight_float <= 100:
                        weights[ticker] = weight_float / 100  # Convert to decimal
                except ValueError:
                    continue

        return weights

    def add_custom_prompt(self, name: str, prompt: str):
        """Add a custom prompt template"""
        self.custom_prompts[name] = prompt


class LLMCouncil:
    """Main LLM Council coordinating multiple members"""

    def __init__(self, api_key: str = None, config: Dict = None):
        self.api_key = api_key
        self.config = config or {}
        self.members = {}
        self.prompt_engineer = PromptEngineer()
        self.consensus_results = {}
        self.cost_tracker = {
            "total_cost": 0.0,
            "total_tokens": 0,
            "total_queries": 0
        }

        # Initialize members from config
        self._initialize_members()

    def _initialize_members(self):
        """Initialize council members from config"""
        for model_config in self.config.get("models", []):
            if model_config.get("enabled", False):
                member = LLMCouncilMember(
                    name=model_config["name"],
                    model_id=model_config["id"],
                    weight=model_config.get("weight", 1.0),
                    cost_type=model_config.get("cost", "free")
                )
                if self.api_key:
                    member.initialize_client(self.api_key)
                self.members[model_config["name"]] = member

    def add_member(self, name: str, model_id: str, weight: float = 1.0, cost_type: str = "free"):
        """Add a new member to the council"""
        member = LLMCouncilMember(name, model_id, weight, cost_type)
        if self.api_key:
            member.initialize_client(self.api_key)
        self.members[name] = member

    def remove_member(self, name: str):
        """Remove a member from the council"""
        if name in self.members:
            del self.members[name]

    def query_all_members(self, prompt: str, temperature: float = 0.3,
                         max_tokens: int = 500) -> Dict[str, LLMResponse]:
        """Query all council members with the same prompt"""
        responses = {}

        print(f"🤖 Querying {len(self.members)} council members...")

        for member_name, member in self.members.items():
            print(f"  • {member_name}...", end="")
            response = member.query(prompt, temperature, max_tokens)
            responses[member_name] = response

            # Update cost tracker
            self.cost_tracker["total_cost"] += response.cost
            self.cost_tracker["total_tokens"] += response.tokens_used
            self.cost_tracker["total_queries"] += 1

            if response.error:
                print(f" ❌ Error: {response.error[:50]}")
            else:
                print(f" ✅ {response.tokens_used} tokens")

            time.sleep(1)  # Rate limiting

        return responses

    def generate_stock_universe(self, num_stocks: int, iterations: int = 2,
                               prompt_type: str = "simple", custom_prompt: str = None) -> Dict:
        """ Generate stock universe through council consensus """
        print(f"🔍 Generating universe of {num_stocks} stocks...")

        prompt = self.prompt_engineer.create_stock_selection_prompt(
            num_stocks, prompt_type, custom_prompt
        )

        all_selections = {}
        frequency_counter = {}

        for iteration in range(iterations):
            print(f"  Iteration {iteration + 1}/{iterations}")

            responses = self.query_all_members(prompt)

            for member_name, response in responses.items():
                if response.error:
                    continue

                tickers = self.prompt_engineer.extract_tickers(response.response)

                if member_name not in all_selections:
                    all_selections[member_name] = []

                # Add unique tickers from this iteration
                for ticker in tickers[:num_stocks]:
                    if ticker not in all_selections[member_name]:
                        all_selections[member_name].append(ticker)

                # Update frequencies
                for ticker in tickers[:num_stocks]:
                    frequency_counter[ticker] = frequency_counter.get(ticker, 0) + 1

        # Get consensus picks (most frequently selected)
        sorted_tickers = sorted(frequency_counter.items(), key=lambda x: x[1], reverse=True)
        consensus_tickers = [ticker for ticker, freq in sorted_tickers[:num_stocks]]

        # Calculate agreement scores
        agreement_scores = {}
        for member_name, selections in all_selections.items():
            common = len(set(selections[:num_stocks]) & set(consensus_tickers))
            agreement_scores[member_name] = common / num_stocks

        result = {
            "individual_selections": {m: s[:num_stocks] for m, s in all_selections.items()},
            "consensus_tickers": consensus_tickers,
            "selection_frequencies": dict(sorted_tickers),
            "agreement_scores": agreement_scores,
            "prompt_used": prompt,
            "total_members": len(self.members),
            "iterations": iterations
        }

        self.consensus_results[f"universe_{num_stocks}"] = result
        return result

    def assign_portfolio_weights(self, tickers: List[str],
                                prompt_type: str = "basic", custom_prompt: str = None) -> Dict:
        """ Get weight assignments from all council members """
        print(f"⚖️ Getting weight assignments for {len(tickers)} stocks...")

        prompt = self.prompt_engineer.create_weight_assignment_prompt(
            tickers, prompt_type, custom_prompt
        )

        responses = self.query_all_members(prompt)

        all_weights = {}
        weight_matrix = {}

        for member_name, response in responses.items():
            if response.error:
                continue

            weights = self.prompt_engineer.extract_weights(response.response)

            # Only include weights for our tickers
            filtered_weights = {t: w for t, w in weights.items() if t in tickers}

            if filtered_weights:
                # Normalize to sum to 1
                total = sum(filtered_weights.values())
                if total > 0:
                    normalized = {t: w/total for t, w in filtered_weights.items()}
                    all_weights[member_name] = normalized

                    # Add to matrix for statistics
                    for ticker, weight in normalized.items():
                        if ticker not in weight_matrix:
                            weight_matrix[ticker] = []
                        weight_matrix[ticker].append(weight)

        # Calculate consensus (average) weights
        consensus_weights = {}
        weight_disagreement = {}

        for ticker in tickers:
            weights_list = weight_matrix.get(ticker, [])
            if weights_list:
                consensus_weights[ticker] = np.mean(weights_list)
                weight_disagreement[ticker] = np.std(weights_list)

        # Normalize consensus weights
        total_consensus = sum(consensus_weights.values())
        if total_consensus > 0:
            consensus_weights = {t: w/total_consensus for t, w in consensus_weights.items()}

        return {
            "individual_weights": all_weights,
            "consensus_weights": consensus_weights,
            "weight_disagreement": weight_disagreement,
            "prompt_used": prompt,
            "total_responses": len(all_weights)
        }

    def get_council_stats(self) -> Dict:
        """Get overall council statistics"""
        stats = {
            "total_members": len(self.members),
            "members": {},
            "total_queries": 0,
            "successful_queries": 0,
            "failed_queries": 0,
            "total_tokens": 0,
            "total_cost": 0.0,
            "free_members": 0,
            "paid_members": 0
        }

        for member_name, member in self.members.items():
            member_stats = member.get_stats()
            stats["members"][member_name] = member_stats
            stats["total_queries"] += member_stats["successful_queries"] + member_stats["failed_queries"]
            stats["successful_queries"] += member_stats["successful_queries"]
            stats["failed_queries"] += member_stats["failed_queries"]
            stats["total_tokens"] += member_stats["total_tokens"]
            stats["total_cost"] += member_stats["total_cost"]

            if member.cost_type == "free":
                stats["free_members"] += 1
            else:
                stats["paid_members"] += 1

        return stats

    def plot_member_performance(self):
        """Visualize council member performance"""
        stats = self.get_council_stats()

        fig, axes = plt.subplots(2, 2, figsize=(14, 10))

        member_names = list(stats["members"].keys())
        colors = plt.cm.Set3(np.linspace(0, 1, len(member_names)))

        # Plot 1: Success Rate
        success_rates = [stats["members"][name]["success_rate"] * 100 for name in member_names]
        bars1 = axes[0, 0].bar(member_names, success_rates, color=colors)
        axes[0, 0].set_title('Query Success Rate (%)', fontsize=12, fontweight='bold')
        axes[0, 0].set_ylabel('Success Rate')
        axes[0, 0].tick_params(axis='x', rotation=45)

        # Add value labels
        for bar, rate in zip(bars1, success_rates):
            axes[0, 0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                          f'{rate:.0f}%', ha='center', va='bottom', fontsize=9)

        # Plot 2: Average Latency
        avg_latency = [stats["members"][name].get("avg_latency", 0) for name in member_names]
        bars2 = axes[0, 1].bar(member_names, avg_latency, color=colors)
        axes[0, 1].set_title('Average Latency (seconds)', fontsize=12, fontweight='bold')
        axes[0, 1].set_ylabel('Seconds')
        axes[0, 1].tick_params(axis='x', rotation=45)

        # Plot 3: Token Usage
        token_usage = [stats["members"][name]["total_tokens"] for name in member_names]
        bars3 = axes[1, 0].bar(member_names, token_usage, color=colors)
        axes[1, 0].set_title('Total Token Usage', fontsize=12, fontweight='bold')
        axes[1, 0].set_ylabel('Tokens')
        axes[1, 0].tick_params(axis='x', rotation=45)

        # Plot 4: Cost by Member
        costs = [stats["members"][name]["total_cost"] for name in member_names]
        bars4 = axes[1, 1].bar(member_names, costs, color=colors)
        axes[1, 1].set_title('Estimated Cost ($)', fontsize=12, fontweight='bold')
        axes[1, 1].set_ylabel('Dollars')
        axes[1, 1].tick_params(axis='x', rotation=45)

        # Add cost labels
        for bar, cost in zip(bars4, costs):
            if cost > 0:
                axes[1, 1].text(bar.get_x() + bar.get_width()/2, bar.get_height(),
                              f'${cost:.4f}', ha='center', va='bottom', fontsize=9)

        plt.suptitle(f'LLM Council Performance (Total Cost: ${stats["total_cost"]:.4f})',
                    fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()

    def plot_stock_selection_heatmap(self, universe_result: Dict):
        """Visualize stock selection heatmap"""
        frequencies = universe_result["selection_frequencies"]

        # Get top 15 stocks
        top_stocks = sorted(frequencies.items(), key=lambda x: x[1], reverse=True)[:15]
        tickers = [s[0] for s in top_stocks]
        freq_values = [s[1] for s in top_stocks]

        # Create member selection matrix
        members = list(universe_result["individual_selections"].keys())
        selection_matrix = []

        for member in members:
            member_selections = universe_result["individual_selections"][member]
            row = [1 if ticker in member_selections else 0 for ticker in tickers]
            selection_matrix.append(row)

        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

        # Plot 1: Selection frequency bar chart
        y_pos = np.arange(len(tickers))
        ax1.barh(y_pos, freq_values, color='steelblue')
        ax1.set_yticks(y_pos)
        ax1.set_yticklabels(tickers)
        ax1.invert_yaxis()
        ax1.set_xlabel('Selection Count')
        ax1.set_title('Stock Selection Frequency', fontsize=12, fontweight='bold')

        # Add count labels
        for i, v in enumerate(freq_values):
            ax1.text(v + 0.1, i, str(v), va='center')

        # Plot 2: Heatmap of member selections
        im = ax2.imshow(selection_matrix, cmap='YlOrRd', aspect='auto')
        ax2.set_xticks(range(len(tickers)))
        ax2.set_xticklabels(tickers, rotation=45, ha='right')
        ax2.set_yticks(range(len(members)))
        ax2.set_yticklabels(members)
        ax2.set_title('Member Selection Matrix', fontsize=12, fontweight='bold')

        # Add text annotations
        for i in range(len(members)):
            for j in range(len(tickers)):
                text = ax2.text(j, i, selection_matrix[i][j],
                              ha="center", va="center", color="black" if selection_matrix[i][j] == 0 else "white")

        plt.colorbar(im, ax=ax2, orientation='vertical', label='Selected (1=Yes, 0=No)')
        plt.tight_layout()
        plt.show()


class LLMCouncilDashboard:
    """Complete Interactive Dashboard with Custom Prompts"""

    def __init__(self):
        self.council = None
        self.data_pipeline = None
        self.optimizer = None
        self.results = {}
        self.output_area = Output()

        self._create_widgets()
        self._setup_dashboard()

    def _create_widgets(self):
        """Create all dashboard widgets"""

        # API Key Section
        self.api_key_input = widgets.Password(
            placeholder='Enter your OpenRouter API key',
            description='🔑 API Key:',
            layout=Layout(width='400px')
        )

        self.api_info = widgets.HTML("""
        <div style=\"background-color: #e8f4f8; padding: 10px; border-radius: 5px; margin: 10px 0;\">
        <h4>💰 API Key Information:</h4>
        <ul>
        <li>You have $10 credit on OpenRouter</li>
        <li>FREE models won't use your credit</li>
        <li>Paid models cost ~$0.001-0.002 per 1000 tokens</li>
        <li>Monitor usage in <a href=\"https://openrouter.ai/account\" target=\"_blank\">OpenRouter Dashboard</a></li>
        </ul>
        </div>
        """)

        # LLM Council Configuration - Now shows ALL 10 models
        self.model_checkboxes = {}
        self.model_weight_sliders = {}

        # Create checkboxes and weight sliders for each model
        for model in config_manager.config["llm"]["models"]:
            model_name = model["name"]
            model_id = model["id"]
            cost_type = model.get("cost", "free")

            # Create checkbox with color coding
            checkbox_color = "green" if cost_type == "free" else "orange" if cost_type == "low" else "red"
            checkbox_label = f"{model_name} ({cost_type})"

            checkbox = widgets.Checkbox(
                value=model["enabled"],
                description=checkbox_label,
                style={'description_color': checkbox_color},
                layout=Layout(width='250px')
            )

            # Create weight slider
            weight_slider = widgets.FloatSlider(
                value=model["weight"],
                min=0.5,
                max=2.0,
                step=0.1,
                description='Weight:',
                layout=Layout(width='200px')
            )

            self.model_checkboxes[model_name] = checkbox
            self.model_weight_sliders[model_name] = weight_slider

        # Group models by cost type
        self.free_models_box = widgets.VBox([
            widgets.HTML("<h4>🆓 FREE Models (No Cost):</h4>"),
            *[widgets.HBox([self.model_checkboxes[m], self.model_weight_sliders[m]])
              for m in self.model_checkboxes if "free" in m.lower() or self._get_cost_type(m) == "free"]
        ])

        self.paid_models_box = widgets.VBox([
            widgets.HTML("<h4>💰 Paid Models (Uses Credit):</h4>"),
            *[widgets.HBox([self.model_checkboxes[m], self.model_weight_sliders[m]])
              for m in self.model_checkboxes if not ("free" in m.lower() or self._get_cost_type(m) == "free")]
        ])

        # Universe Configuration
        self.universe_size_slider = widgets.IntSlider(
            value=10,
            min=5,
            max=30,
            step=5,
            description='Stocks to Select:',
            continuous_update=False,
            layout=Layout(width='400px')
        )

        self.iterations_slider = widgets.IntSlider(
            value=2,
            min=1,
            max=5,
            description='Iterations:',
            layout=Layout(width='300px')
        )

        # Prompt Configuration - ADDED CUSTOM PROMPT OPTIONS
        self.prompt_type_dropdown = widgets.Dropdown(
            options=['simple', 'detailed', 'thematic', 'custom'],
            value='simple',
            description='Prompt Type:',
            layout=Layout(width='200px')
        )

        self.custom_prompt_text = widgets.Textarea(
            value='Select {num_stocks} stocks from the S&P 500 that you believe will perform well.\nReturn ONLY ticker symbols separated by commas.',
            placeholder='Enter your custom prompt here... Use {num_stocks} placeholder.',
            description='Custom Prompt:',
            layout=Layout(width='500px', height='100px'),
            style={'description_width': '100px'}
        )

        self.weight_prompt_type = widgets.Dropdown(
            options=['basic', 'advanced', 'custom'],
            value='basic',
            description='Weight Prompt:',
            layout=Layout(width='200px')
        )

        self.custom_weight_prompt = widgets.Textarea(
            value='Given these stocks: {tickers}\nAssign portfolio weights (0-100%) that sum to 100%.',
            placeholder='Enter custom weight prompt... Use {tickers} placeholder.',
            description='Custom Weight:',
            layout=Layout(width='500px', height='80px'),
            style={'description_width': '100px'}
        )

        # Optimization Strategies
        self.strategy_checkboxes = {}
        for strategy in config_manager.config["optimization"]["strategies"]:
            checkbox = widgets.Checkbox(
                value=strategy["enabled"],
                description=strategy["name"],
                layout=Layout(width='150px')
            )
            self.strategy_checkboxes[strategy["name"]] = checkbox

        # Action Buttons
        button_layout = Layout(width='180px', height='40px', margin='5px')

        self.initialize_button = widgets.Button(
            description='🤖 Initialize Council',
            button_style='primary',
            layout=button_layout
        )

        self.test_button = widgets.Button(
            description='🔌 Test Connection',
            button_style='info',
            layout=button_layout
        )

        self.run_button = widgets.Button(
            description='🚀 Run Analysis',
            button_style='success',
            layout=button_layout
        )

        self.clear_button = widgets.Button(
            description='🗑️ Clear Results',
            button_style='warning',
            layout=button_layout
        )

        self.save_button = widgets.Button(
            description='💾 Save Config',
            button_style='',
            layout=button_layout
        )

        # Connect button callbacks
        self.initialize_button.on_click(self._initialize_council)
        self.test_button.on_click(self._test_connection)
        self.run_button.on_click(self._run_analysis)
        self.clear_button.on_click(self._clear_results)
        self.save_button.on_click(self._save_config)

    def _get_cost_type(self, model_name: str) -> str:
        """Get cost type for a model"""
        for model in config_manager.config["llm"]["models"]:
            if model["name"] == model_name:
                return model.get("cost", "free")
        return "free"

    def _setup_dashboard(self):
        """Setup the dashboard layout"""

        # Main title
        title_html = widgets.HTML("""
        <div style=\"text-align: center; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);\n                    color: white; padding: 25px; border-radius: 10px; margin-bottom: 20px;\">
        <h1 style=\"margin: 0; font-size: 2.5em;\">🏛️ LLM Council Portfolio Optimizer</h1>\n        <p style=\"margin: 10px 0; font-size: 1.2em;\">AI-Augmented Investment Strategy with 10+ LLM Models</p>\n        <p style=\"margin: 5px 0; font-size: 1em;\">Configure your council of 5+ AI analysts with custom prompts</p>\n        </div>
        """)

        # Configuration Panel
        config_panel = widgets.VBox([
            widgets.HTML("<h2>⚙️ Configuration Panel</h2>"),

            widgets.HTML("<h3>🔑 API Configuration</h3>"),
            self.api_key_input,
            self.api_info, # Changed api_info to self.api_info
            widgets.HBox([self.initialize_button, self.test_button]),

            widgets.HTML("<h3>🤖 LLM Council (Select 5+ Members)</h3>"),
            widgets.HTML("<p>Select which AI models to include in your council. Green = Free, Orange/Red = Paid</p>"),
            widgets.HBox([self.free_models_box, self.paid_models_box]),

            widgets.HTML("<h3>📊 Stock Selection</h3>"),
            widgets.HBox([self.universe_size_slider, self.iterations_slider]),

            widgets.HTML("<h3>💬 Prompt Configuration</h3>"),
            widgets.VBox([
                widgets.HBox([self.prompt_type_dropdown, widgets.HTML("<b>Stock Selection Prompt</b>")]),
                self.custom_prompt_text,
                widgets.HBox([self.weight_prompt_type, widgets.HTML("<b>Weight Assignment Prompt</b>")]),
                self.custom_weight_prompt
            ]),

            widgets.HTML("<h3>⚙️ Optimization Strategies</h3>"),
            widgets.HBox(list(self.strategy_checkboxes.values())),

            widgets.HTML("<h3>🚀 Actions</h3>"),
            widgets.HBox([self.run_button, self.clear_button, self.save_button])
        ], layout=Layout(border='2px solid #ddd', padding='15px', width='50%'))

        # Results Panel
        self.results_panel = widgets.VBox([
            widgets.HTML("<h2>📊 Analysis Results</h2>"),
            self.output_area
        ], layout=Layout(border='2px solid #ddd', padding='15px', width='50%'))

        # Main Dashboard Layout
        self.dashboard = widgets.VBox([
            title_html,
            widgets.HBox([config_panel, self.results_panel])
        ])

    def _initialize_council(self, b):
        """Initialize the LLM Council"""
        with self.output_area:
            clear_output()

            if not self.api_key_input.value:
                print("❌ Please enter your OpenRouter API key")
                print("   Get your key from: https://openrouter.ai/account")
                return

            # Count enabled models
            enabled_count = sum(1 for checkbox in self.model_checkboxes.values() if checkbox.value)
            if enabled_count < 5:
                print(f"⚠️  Only {enabled_count} models selected. For best results, select at least 5 council members.")
                response = input("Continue anyway? (y/n): ")
                if response.lower() != 'y':
                    return

            print("🔄 Initializing LLM Council...")

            # Update config with widget values
            for model_name, checkbox in self.model_checkboxes.items():
                config_manager.update_model_status(model_name, checkbox.value)
                # Update weight if slider exists
                if model_name in self.model_weight_sliders:
                    for model in config_manager.config["llm"]["models"]:
                        if model["name"] == model_name:
                            model["weight"] = self.model_weight_sliders[model_name].value

            # Create LLM Council
            self.council = LLMCouncil(
                api_key=self.api_key_input.value,
                config=config_manager.config["llm"]
            )

            # Count free vs paid models
            free_count = sum(1 for m in self.council.members.values() if m.cost_type == "free")
            paid_count = len(self.council.members) - free_count

            print("✅ LLM Council initialized successfully!")
            print(f"   Total members: {len(self.council.members)}")
            print(f"   Free models: {free_count}, Paid models: {paid_count}")
            print(f"   Members: {', '.join(self.council.members.keys())}")

            if paid_count > 0:
                print(f"   ⚠️  Using {paid_count} paid models. Monitor credit usage at: https://openrouter.ai/account")

    def _test_connection(self, b):
        """Test connection to OpenRouter"""
        with self.output_area:
            clear_output()

            if not self.api_key_input.value:
                print("❌ Please enter your OpenRouter API key")
                return

            print("🔌 Testing connection to OpenRouter...")

            try:
                client = openai.OpenAI(
                    base_url="https://openrouter.ai/api/v1",
                    api_key=self.api_key_input.value
                )

                # Test with a free model
                response = client.chat.completions.create(
                    model="meta-llama/llama-3.2-3b-instruct:free", # Changed to Llama 3.2 3B free model
                    messages=[{"role": "user", "content": "Say 'Connection successful'"}],
                    max_tokens=10
                )

                print(f"✅ Connection successful!")
                print(f"   Model: {response.model}")
                print(f"   Response: {response.choices[0].message.content}")
                print(f"   Tokens used: {response.usage.total_tokens}")

                # Also check account balance
                print("\n💰 Checking account status...")
                print("   Note: For detailed balance, visit: https://openrouter.ai/account")
                print("   You have $10 credit available for paid models.")

            except Exception as e:
                print(f"❌ Connection failed: {e}")
                print("\n💡 Troubleshooting tips:")
                print("1. Check if API key is correct")
                print("2. Visit https://openrouter.ai/account to verify key")
                print("3. Ensure you have credit balance")
                print("4. Try using only FREE models first")

    def _run_analysis(self, b):
        """Run the complete analysis pipeline"""
        with self.output_area:
            clear_output()

            if not self.council:
                print("❌ Please initialize the LLM Council first")
                return

            if len(self.council.members) == 0:
                print("❌ No LLM models enabled. Please select at least one model.")
                return

            print("🚀 Starting Complete Analysis Pipeline")
            print("=" * 60)

            try:
                # Determine which prompt to use
                prompt_type = self.prompt_type_dropdown.value
                custom_prompt = self.custom_prompt_text.value if prompt_type == "custom" else None

                # Step 1: Generate stock universe
                print("\n📈 STEP 1: Stock Selection with LLM Council")
                print("-" * 40)

                universe_result = self.council.generate_stock_universe(
                    num_stocks=self.universe_size_slider.value,
                    iterations=self.iterations_slider.value,
                    prompt_type=prompt_type,
                    custom_prompt=custom_prompt
                )

                print(f"   ✅ Selected {len(universe_result['consensus_tickers'])} consensus stocks")
                print(f"   Top 5 picks: {', '.join(universe_result['consensus_tickers'][:5])}")

                # Display selection statistics
                if "agreement_scores" in universe_result:
                    print("\n   🤝 Agreement Scores:")
                    for member, score in universe_result["agreement_scores"].items():
                        print(f"      {member}: {score:.1%}")

                # Step 2: Get weight assignments
                print("\n⚖️ STEP 2: Portfolio Weight Assignment")
                print("-" * 40)

                weight_prompt_type = self.weight_prompt_type.value
                custom_weight_prompt = self.custom_weight_prompt.value if weight_prompt_type == "custom" else None

                weight_result = self.council.assign_portfolio_weights(
                    universe_result["consensus_tickers"],
                    prompt_type=weight_prompt_type,
                    custom_prompt=custom_weight_prompt
                )

                print(f"   ✅ Got weights from {weight_result['total_responses']} council members")

                # Step 3: Prepare market data
                print("\n📊 STEP 3: Market Data Preparation")
                print("-" * 40)

                self.data_pipeline = DataPipeline(
                    start_date="2021-01-01",
                    end_date="2024-01-31"
                )

                portfolio_data = self.data_pipeline.prepare_portfolio_data(
                    universe_result["consensus_tickers"],
                    split_date="2023-01-01"
                )

                print(f"   ✅ Loaded data for {len(portfolio_data['tickers'])} stocks")
                print(f"   Period: {portfolio_data['returns'].index[0].date()} to {portfolio_data['returns'].index[-1].date()}")

                # Step 4: Portfolio Optimization
                print("\n🔄 STEP 4: Portfolio Optimization")
                print("-" * 40)

                # Get enabled strategies
                enabled_strategies = []
                for name, checkbox in self.strategy_checkboxes.items():
                    if checkbox.value:
                        enabled_strategies.append(name)

                if not enabled_strategies:
                    print("   ⚠️  No optimization strategies selected. Using default strategies.")
                    enabled_strategies = ["Equal Weight", "LLM Weighted"]

                print(f"   Running {len(enabled_strategies)} strategies: {', '.join(enabled_strategies)}")

                self.optimizer = PortfolioOptimizer()
                strategies = self.optimizer.optimize_all_strategies(
                    tickers=portfolio_data['tickers'],
                    expected_returns=portfolio_data['expected_returns'],
                    covariance_matrix=portfolio_data['covariance_matrix'],
                    llm_weights=weight_result["consensus_weights"]
                )

                # Filter to only enabled strategies
                filtered_strategies = {name: strat for name, strat in strategies.items()
                                     if name.lower().replace(" ", "_") in [s.lower().replace(" ", "_") for s in enabled_strategies]}

                # Store results
                self.results = {
                    'universe': universe_result,
                    'weights': weight_result,
                    'strategies': filtered_strategies,
                    'portfolio_data': portfolio_data,
                    'council_stats': self.council.get_council_stats()
                }

                print("\n✅ ANALYSIS COMPLETE!")
                print("=" * 60)

                # Display summary
                council_stats = self.council.get_council_stats()
                print(f"\n📋 SUMMARY:")
                print(f"   • Council Members: {council_stats['total_members']}")
                print(f"   • Total Queries: {council_stats['total_queries']}")
                print(f"   • Total Tokens: {council_stats['total_tokens']:,}")
                print(f"   • Estimated Cost: ${council_stats['total_cost']:.4f}")
                print(f"   • Stocks Selected: {len(universe_result['consensus_tickers'])}")
                print(f"   • Strategies Tested: {len(filtered_strategies)}")

                # Display results
                self._display_results()

            except Exception as e:
                print(f"\n❌ Error during analysis: {e}")
                import traceback
                traceback.print_exc()
                print("\n💡 Try these fixes:")
                print("1. Check your API key and credit balance")
                print("2. Reduce number of stocks or iterations")
                print("3. Try using only FREE models")
                print("4. Check internet connection")

    def _display_results(self):
        """Display analysis results in tabs"""
        if not self.results:
            return

        # Create tabs
        tab_contents = []
        tab_titles = []

        # Tab 1: Council Performance
        tab1_output = Output()
        with tab1_output:
            if self.council:
                # Display council stats
                stats = self.results.get('council_stats', {})
                print("🤖 LLM COUNCIL PERFORMANCE")
                print("=" * 40)
                print(f"Total Members: {stats.get('total_members', 0)}")
                print(f"Free Members: {stats.get('free_members', 0)}")
                print(f"Paid Members: {stats.get('paid_members', 0)}")
                print(f"Total Queries: {stats.get('total_queries', 0)}")
                print(f"Success Rate: {stats.get('successful_queries', 0)/max(1, stats.get('total_queries', 1)):.1%}")
                print(f"Total Tokens: {stats.get('total_tokens', 0):,}")
                print(f"Total Cost: ${stats.get('total_cost', 0):.4f}")

                # Plot performance
                self.council.plot_member_performance()

                # Show stock selection heatmap
                if 'universe' in self.results:
                    self.council.plot_stock_selection_heatmap(self.results['universe'])

        tab_contents.append(tab1_output)
        tab_titles.append("Council")

        # Tab 2: Stock Selection
        tab2_output = Output()
        with tab2_output:
            if 'universe' in self.results:
                universe = self.results['universe']

                print("🎯 STOCK SELECTION RESULTS")
                print("=" * 40)
                print(f"Consensus Stocks ({len(universe['consensus_tickers'])}):")

                # Create DataFrame for display
                selection_data = []
                for i, ticker in enumerate(universe['consensus_tickers'], 1):
                    freq = universe['selection_frequencies'].get(ticker, 0)
                    selection_data.append({
                        '#': i,
                        'Ticker': ticker,
                        'Selection Count': freq,
                        'Selected By': f"{freq}/{universe['total_members'] * universe['iterations']}"
                    })

                selection_df = pd.DataFrame(selection_data)
                display(selection_df)

                # Display individual member selections
                print("\n📝 Individual Member Selections:")
                for member, picks in universe['individual_selections'].items():
                    print(f"   {member}: {', '.join(picks[:5])}...")

        tab_contents.append(tab2_output)
        tab_titles.append("Stocks")

        # Tab 3: Portfolio Strategies
        tab3_output = Output()
        with tab3_output:
            if 'strategies' in self.results:
                print("📊 PORTFOLIO STRATEGY COMPARISON")
                print("=" * 40)

                # Create comparison table
                comparison_data = []
                for name, strategy in self.results['strategies'].items():
                    comparison_data.append({
                        'Strategy': name.replace('_', ' ').title(),
                        'Type': strategy['type'],
                        'Expected Return': f"{strategy['expected_return']:.1%}",
                        'Volatility': f"{strategy['volatility']:.1%}",
                        'Sharpe Ratio': f"{strategy['sharpe_ratio']:.2f}",
                        'Top Holdings': self._get_top_holdings(strategy['weights'],
                                                            self.results['portfolio_data']['tickers'])
                    })

                comparison_df = pd.DataFrame(comparison_data)
                display(comparison_df)

                # Plot portfolio weights
                self._plot_portfolio_comparison()

        tab_contents.append(tab3_output)
        tab_titles.append("Portfolios")

        # Tab 4: Stock Performance
        tab4_output = Output()
        with tab4_output:
            if 'portfolio_data' in self.results:
                print("📈 STOCK PERFORMANCE ANALYSIS")
                print("=" * 40)

                portfolio_data = self.results['portfolio_data']
                returns = portfolio_data['in_sample_returns']

                # Calculate metrics for each stock
                metrics_data = []
                for ticker in portfolio_data['tickers'][:15]:  # Show first 15
                    stock_returns = returns[ticker]
                    if len(stock_returns) > 10:
                        ann_return = stock_returns.mean() * 52
                        ann_vol = stock_returns.std() * np.sqrt(52)
                        sharpe = (ann_return - 0.02) / ann_vol if ann_vol > 0 else 0

                        metrics_data.append({
                            'Ticker': ticker,
                            'Ann. Return': f"{ann_return:.1%}",
                            'Ann. Volatility': f"{ann_vol:.1%}",
                            'Sharpe Ratio': f"{sharpe:.2f}",
                            'Max Return': f"{stock_returns.max():.1%}",
                            'Min Return': f"{stock_returns.min():.1%}"
                        })

                metrics_df = pd.DataFrame(metrics_data)
                display(metrics_df)

                # Plot price evolution
                self._plot_price_performance()

        tab_contents.append(tab4_output)
        tab_titles.append("Performance")

        # Tab 5: Weights & Allocation
        tab5_output = Output()
        with tab5_output:
            if 'weights' in self.results:
                print("⚖️ PORTFOLIO WEIGHTS ASSIGNMENT")
                print("=" * 40)

                weights = self.results['weights']

                # Display consensus weights
                if 'consensus_weights' in weights:
                    print("Consensus Weights:")
                    weight_data = []
                    for ticker, weight in weights['consensus_weights'].items():
                        weight_data.append({
                            'Ticker': ticker,
                            'Weight': f"{weight:.1%}",
                            'Disagreement': f"{weights['weight_disagreement'].get(ticker, 0):.3f}"
                            if 'weight_disagreement' in weights else 'N/A'
                        })

                    weight_df = pd.DataFrame(weight_data).sort_values('Weight', ascending=False)
                    display(weight_df)

                # Display individual member weights
                if 'individual_weights' in weights and weights['individual_weights']:
                    print("\nIndividual Member Weights (Top 3 per member):")
                    for member, member_weights in weights['individual_weights'].items():
                        sorted_weights = sorted(member_weights.items(), key=lambda x: x[1], reverse=True)[:3]
                        top_str = ', '.join([f"{t}: {w:.1%}" for t, w in sorted_weights])
                        print(f"   {member}: {top_str}")

        tab_contents.append(tab5_output)
        tab_titles.append("Weights")

        # Display tabs
        tabs = widgets.Tab(children=tab_contents)
        for i, title in enumerate(tab_titles):
            tabs.set_title(i, title)

        display(tabs)

    def _get_top_holdings(self, weights: np.ndarray, tickers: List[str], n: int = 3) -> str:
        """Get top n holdings as string"""
        sorted_indices = np.argsort(weights)[::-1][:n]
        top_tickers = [tickers[i] for i in sorted_indices]
        top_weights = weights[sorted_idx] * 100
        return ', '.join([f"{t}: {w:.1f}%" for t, w in zip(top_tickers, top_weights)])

    def _plot_portfolio_comparison(self):
        """Plot portfolio strategy comparison"""
        if 'strategies' not in self.results:
            return

        strategies = self.results['strategies']

        fig, axes = plt.subplots(2, 2, figsize=(14, 10))

        # Prepare data
        strategy_names = list(strategies.keys())
        returns = [strategies[name]['expected_return'] for name in strategy_names]
        volatilities = [strategies[name]['volatility'] for name in strategy_names]
        sharpes = [strategies[name]['sharpe_ratio'] for name in strategy_names]

        colors = plt.cm.viridis(np.linspace(0, 1, len(strategy_names)))

        # Plot 1: Return vs Risk
        ax1 = axes[0, 0]
        scatter1 = ax1.scatter(volatilities, returns, c=colors, s=200, alpha=0.7)

        # Add labels
        for i, name in enumerate(strategy_names):
            ax1.annotate(name, (volatilities[i], returns[i]),
                        xytext=(5, 5), textcoords='offset points', fontsize=9)

        ax1.set_xlabel('Volatility (Risk)')
        ax1.set_ylabel('Expected Return')
        ax1.set_title('Risk-Return Profile')
        ax1.grid(True, alpha=0.3)

        # Plot 2: Sharpe Ratios
        ax2 = axes[0, 1]
        bars = ax2.barh(strategy_names, sharpes, color=colors)
        ax2.set_xlabel('Sharpe Ratio')
        ax2.set_title('Risk-Adjusted Performance')
        ax2.grid(True, alpha=0.3, axis='x')

        # Add value labels
        for bar, sharpe in zip(bars, sharpes):
            ax2.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height()/2,
                    f'{sharpe:.2f}', va='center', fontsize=9)

        # Plot 3: Weight Distribution (for first strategy)d
        ax3 = axes[1, 0]
        if strategy_names:
            first_strategy = strategies[strategy_names[0]]
            weights = first_strategy['weights']
            tickers = self.results['portfolio_data']['tickers']

            # Get top 10 holdings
            sorted_idx = np.argsort(weights)[::-1][:10]
            top_tickers = [tickers[i] for i in sorted_idx]
            top_weights = weights[sorted_idx] * 100

            bars3 = ax3.barh(range(len(top_tickers)), top_weights, color=colors[:len(top_tickers)])
            ax3.set_yticks(range(len(top_tickers)))
            ax3.set_yticklabels(top_tickers)
            ax3.set_xlabel('Weight (%)')
            ax3.set_title(f'Top Holdings: {strategy_names[0]}')
            ax3.invert_yaxis()

            # Add weight labels
            for bar, weight in zip(bars3, top_weights):
                ax3.text(bar.get_width() + 0.5, bar.get_y() + bar.get_height()/2,
                        f'{weight:.1f}%', va='center', fontsize=9)

        # Plot 4: Strategy Comparison Radar
        ax4 = axes[1, 1]
        # Normalize metrics for radar plot
        norm_returns = (returns - np.min(returns)) / (np.max(returns) - np.min(returns) + 1e-10)
        norm_sharpes = (sharpes - np.min(sharpes)) / (np.max(sharpes) - np.min(sharpes) + 1e-10)
        inv_vols = 1 / (np.array(volatilities) + 1e-10)
        norm_inv_vols = (inv_vols - np.min(inv_vols)) / (np.max(inv_vols) - np.min(inv_vols) + 1e-10)

        angles = np.linspace(0, 2*np.pi, 3, endpoint=False).tolist()
        angles += angles[:1]  # Close the polygon

        for i, name in enumerate(strategy_names):
            values = [norm_returns[i], norm_sharpes[i], norm_inv_vols[i]]
            values += values[:1]  # Close the polygon
            ax4.plot(angles, values, 'o-', label=name, linewidth=2, markersize=6)
            ax4.fill(angles, values, alpha=0.1)

        ax4.set_xticks(angles[:-1])
        ax4.set_xticklabels(['Return', 'Sharpe', 'Low Risk'])
        ax4.set_title('Strategy Comparison')
        ax4.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0), fontsize=9)

        plt.suptitle('Portfolio Strategy Analysis', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()

    def _plot_price_performance(self):
        """Plot stock price performance"""
        if 'portfolio_data' not in self.results:
            return

        portfolio_data = self.results['portfolio_data']
        prices = portfolio_data['prices']

        # Normalize prices to start at 100
        normalized_prices = prices / prices.iloc[0] * 100

        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

        # Plot 1: Individual stock performance
        tickers = portfolio_data['tickers'][:8]  # Show first 8
        for ticker in tickers:
            if ticker in normalized_prices.columns:
                ax1.plot(normalized_prices.index, normalized_prices[ticker],
                        label=ticker, linewidth=2, alpha=0.8)

        ax1.set_title('Stock Performance (Normalized to 100)', fontsize=12, fontweight='bold')
        ax1.set_xlabel('Date')
        ax1.set_ylabel('Normalized Price')
        ax1.legend(loc='upper left', bbox_to_anchor=(1, 1))
        ax1.grid(True, alpha=0.3)

        # Plot 2: Cumulative returns
        returns = portfolio_data['returns'][tickers]
        cumulative_returns = (1 + returns).cumprod() - 1

        for ticker in tickers:
            if ticker in cumulative_returns.columns:
                ax2.plot(cumulative_returns.index, cumulative_returns[ticker] * 100,
                        label=ticker, linewidth=2, alpha=0.8)

        ax2.set_title('Cumulative Returns (%)', fontsize=12, fontweight='bold')
        ax2.set_xlabel('Date')
        ax2.set_ylabel('Cumulative Return (%)')
        ax2.legend(loc='upper left', bbox_to_anchor=(1, 1))
        ax2.grid(True, alpha=0.3)

        plt.tight_layout()
        plt.show()

    def _clear_results(self, b):
        """Clear all results"""
        with self.output_area:
            clear_output()
        self.results = {}
        print("🗑️ Results cleared. Ready for new analysis.")

    def _save_config(self, b):
        """Save current configuration"""
        # Update config with current widget values
        for model_name, checkbox in self.model_checkboxes.items():
            config_manager.update_model_status(model_name, checkbox.value)
            if model_name in self.model_weight_sliders:
                for model in config_manager.config["llm"]["models"]:
                    if model["name"] == model_name:
                        model["weight"] = self.model_weight_sliders[model_name].value

        # Save to file
        config_file = "llm_council_config.json"
        config_manager.save_config(config_file)

        with self.output_area:
            clear_output()
            print(f"✅ Configuration saved to {config_file}")
            print("\nCurrent Configuration:")
            print(f"  • Selected models: {sum(1 for cb in self.model_checkboxes.values() if cb.value)}")
            print(f"  • Universe size: {self.universe_size_slider.value}")
            print(f"  • Iterations: {self.iterations_slider.value}")
            print(f"  • Prompt type: {self.prompt_type_dropdown.value}")

    def display(self):
        """Display the dashboard"""
        display(self.dashboard)


class DataPipeline:
    """Handle all data collection and processing"""

    def __init__(self, start_date: str, end_date: str):
        self.start_date = pd.to_datetime(start_date)
        self.end_date = pd.to_datetime(end_date)

    def fetch_stock_data(self, tickers: List[str]) -> pd.DataFrame:
        """Fetch historical stock data"""
        print(f"📊 Fetching data for {len(tickers)} stocks...")

        all_data = {}
        valid_tickers = []

        for ticker in tqdm(tickers, desc="Downloading"):
            try:
                stock = yf.Ticker(ticker)
                data = stock.history(start=self.start_date, end=self.end_date, interval='1wk')

                if not data.empty and len(data) > 20:
                    all_data[ticker] = data['Close']
                    valid_tickers.append(ticker)

                time.sleep(0.1)

            except:
                continue

        price_df = pd.DataFrame(all_data)

        # Handle missing values
        price_df = price_df.ffill().bfill()

        return price_df

    def prepare_portfolio_data(self, tickers: List[str], split_date: str) -> Dict:
        """Prepare complete dataset for portfolio analysis"""
        # Fetch price data
        price_df = self.fetch_stock_data(tickers)

        if price_df.empty:
            raise ValueError("No data available for selected tickers")

        # Calculate returns
        returns = price_df.pct_change().dropna()

        # Split data
        split_date = pd.to_datetime(split_date)

        # Ensure timezone consistency
        if returns.index.tz is not None and split_date.tz is None:
            split_date = split_date.tz_localize(returns.index.tz)
        elif returns.index.tz is None and split_date.tz is not None:
            returns.index = returns.index.tz_localize(split_date.tz)
        elif returns.index.tz is not None and split_date.tz is not None and returns.index.tz != split_date.tz:
            split_date = split_date.tz_convert(returns.index.tz) # Convert split_date to match returns.index timezone

        in_sample = returns[returns.index < split_date]
        out_of_sample = returns[returns.index >= split_date]

        # Calculate moments
        expected_returns = in_sample.mean() * 52
        cov_matrix = in_sample.cov() * 52

        # Get benchmark
        benchmark = yf.Ticker('SPY')
        benchmark_prices = benchmark.history(start=self.start_date, end=self.end_date, interval='1wk')['Close']
        benchmark_returns = benchmark_prices.pct_change().dropna()

        return {
            'tickers': price_df.columns.tolist(),
            'prices': price_df,
            'returns': returns,
            'in_sample_returns': in_sample,
            'out_of_sample_returns': out_of_sample,
            'expected_returns': expected_returns,
            'covariance_matrix': cov_matrix,
            'benchmark_returns': benchmark_returns
        }


class PortfolioOptimizer:
    """Implement portfolio optimization strategies"""

    def __init__(self, risk_free_rate: float = 0.02):
        self.risk_free_rate = risk_free_rate

    def equal_weight(self, n_assets: int) -> np.ndarray:
        """Equal weight portfolio"""
        return np.ones(n_assets) / n_assets

    def mean_variance(self, expected_returns: np.ndarray,
                     covariance_matrix: np.ndarray) -> np.ndarray:
        """Mean-variance optimization"""
        n = len(expected_returns)

        # Simple implementation
        volatilities = np.sqrt(np.diag(covariance_matrix))
        # Use return/volatility ratio for weights
        ratios = expected_returns / (volatilities + 1e-10)
        weights = ratios / ratios.sum()

        return weights

    def max_sharpe(self, expected_returns: np.ndarray,
                  covariance_matrix: np.ndarray) -> np.ndarray:
        """Maximum Sharpe ratio portfolio"""
        n = len(expected_returns)

        # Simple heuristic: weight by Sharpe ratio
        volatilities = np.sqrt(np.diag(covariance_matrix))
        excess_returns = expected_returns - self.risk_free_rate
        sharpes = excess_returns / (volatilities + 1e-10)

        # Use softmax for weights
        exp_sharpes = np.exp(sharpes - np.max(sharpes))
        weights = exp_sharpes / exp_sharpes.sum()

        return weights

    def min_variance(self, covariance_matrix: np.ndarray) -> np.ndarray:
        """Minimum variance portfolio"""
        n = covariance_matrix.shape[0]
        volatilities = np.sqrt(np.diag(covariance_matrix))

        # Inverse volatility weighting
        inv_vol = 1 / (volatilities + 1e-10)
        weights = inv_vol / inv_vol.sum()

        return weights

    def risk_parity(self, covariance_matrix: np.ndarray) -> np.ndarray:
        """Risk parity portfolio"""
        return self.min_variance(covariance_matrix)

    def optimize_all_strategies(self, tickers: List[str],
                               expected_returns: pd.Series,
                               covariance_matrix: pd.DataFrame,
                               llm_weights: Dict = None) -> Dict[str, Dict]:
        """Run all optimization strategies"""
        n = len(tickers)
        er_array = expected_returns.values
        cov_array = covariance_matrix.values

        strategies = {
            "equal_weight": {
                "weights": self.equal_weight(n),
                "type": "naive"
            },
            "mean_variance": {
                "weights": self.mean_variance(er_array, cov_array),
                "type": "optimization"
            },
            "max_sharpe": {
                "weights": self.max_sharpe(er_array, cov_array),
                "type": "optimization"
            },
            "min_variance": {
                "weights": self.min_variance(cov_array),
                "type": "optimization"
            },
            "risk_parity": {
                "weights": self.risk_parity(cov_array),
                "type": "risk_based"
            }
        }

        # Add LLM-weighted portfolio
        if llm_weights:
            llm_weight_array = np.zeros(n)
            for i, ticker in enumerate(tickers):
                llm_weight_array[i] = llm_weights.get(ticker, 0)

            if llm_weight_array.sum() > 0:
                llm_weight_array = llm_weight_array / llm_weight_array.sum()
                strategies["llm_weighted"] = {
                    "weights": llm_weight_array,
                    "type": "llm"
                }

        # Calculate metrics
        for name, strategy in strategies.items():
            weights = strategy["weights"]

            portfolio_return = er_array @ weights
            portfolio_risk = np.sqrt(weights.T @ cov_array @ weights)
            sharpe = (portfolio_return - self.risk_free_rate) / portfolio_risk if portfolio_risk > 0 else 0

            strategy.update({
                "expected_return": portfolio_return,
                "volatility": portfolio_risk,
                "sharpe_ratio": sharpe
            })

        return strategies


def main():
    """Main execution function"""

    print("""
    🏛️ LLM COUNCIL PORTFOLIO OPTIMIZER
    ==================================

    Welcome to the AI-Augmented Investment Platform!

    🔑 You have: $10 OpenRouter credit + FREE models

    Features:
    • 10+ LLM models (5 FREE, 5 Paid)
    • Custom prompt engineering
    • 5+ council member configuration
    • Multiple optimization strategies
    • Comprehensive performance analysis

    How to use:
    1. Enter your OpenRouter API key
    2. Select 5+ AI models for your council
    3. Configure stock selection parameters
    4. Customize prompts (optional)
    5. Click 'Initialize Council'
    6. Click 'Run Analysis'

    💡 Tips:
    • Start with FREE models to test
    • Select at least 5 council members
    • Monitor credit usage at: https://openrouter.ai/account
    • Custom prompts can improve results

    """)

    # Create and display dashboard
    dashboard = LLMCouncilDashboard()
    dashboard.display()


if __name__ == "__main__":
    main()

def quick_start_examples():
    """Show quick start examples"""

    print("🚀 QUICK START EXAMPLES")
    print("=" * 50)

    print("\n1️⃣ Minimal Setup (FREE only):")
    print("""
    Configuration:
    • Models: All 5 FREE models (Llama, Mistral, Gemma, Qwen, Phi-3)
    • Universe: 10 stocks
    • Iterations: 2
    • Prompts: Default simple prompts
    • Cost: $0.00
    """)

    print("\n2️⃣ Balanced Setup (Mixed FREE & Paid):")
    print("""
    Configuration:
    • Models: 3 FREE + 2 Paid (GPT-3.5 + Claude Haiku)
    • Universe: 15 stocks
    • Iterations: 3
    • Prompts: Detailed prompts
    • Estimated cost: ~$0.02
    """)

    print("\n3️⃣ Advanced Setup (Full Council):")
    print("""
    Configuration:
    • Models: All 10 models enabled
    • Universe: 20 stocks
    • Iterations: 3
    • Prompts: Custom tailored prompts
    • Estimated cost: ~$0.05-0.10
    """)

    print("\n💡 Your $10 credit can run:")
    print("• 200+ Minimal setups")
    print("• 50+ Balanced setups")
    print("• 10-20 Advanced setups")

    return True


print("Launching LLM Council Portfolio Optimizer...")
print("\n" + "="*60)
main()

✅ Packages installed and imported successfully!

    🏛️ LLM COUNCIL PORTFOLIO OPTIMIZER

    Welcome to the AI-Augmented Investment Platform!

    🔑 You have: $10 OpenRouter credit + FREE models

    Features:
    • 10+ LLM models (5 FREE, 5 Paid)
    • Custom prompt engineering
    • 5+ council member configuration
    • Multiple optimization strategies
    • Comprehensive performance analysis

    How to use:
    1. Enter your OpenRouter API key
    2. Select 5+ AI models for your council
    3. Configure stock selection parameters
    4. Customize prompts (optional)
    5. Click 'Initialize Council'
    6. Click 'Run Analysis'

    💡 Tips:
    • Start with FREE models to test
    • Select at least 5 council members
    • Monitor credit usage at: https://openrouter.ai/account
    • Custom prompts can improve results

    


Launching LLM Council Portfolio Optimizer...


    🏛️ LLM COUNCIL PORTFOLIO OPTIMIZER

    Welcome to the AI-Augmented Investment Platform!

    🔑 You have: $10 OpenRouter credit + FREE models

    Features:
    • 10+ LLM models (5 FREE, 5 Paid)
    • Custom prompt engineering
    • 5+ council member configuration
    • Multiple optimization strategies
    • Comprehensive performance analysis

    How to use:
    1. Enter your OpenRouter API key
    2. Select 5+ AI models for your council
    3. Configure stock selection parameters
    4. Customize prompts (optional)
    5. Click 'Initialize Council'
    6. Click 'Run Analysis'

    💡 Tips:
    • Start with FREE models to test
    • Select at least 5 council members
    • Monitor credit usage at: https://openrouter.ai/account
    • Custom prompts can improve results

    


🗑️ Results cleared. Ready for new analysis.
🗑️ Results cleared. Ready for new analysis.
